In [ ]:
GPe_input = 40:10:100; % pA, inhibitory
for i = 1:length(GPe_input)
%Runs 20s simulation without any stimulus to measure steady-state values. 
[g_gp2snr] = BGdelayline_setinit2('I_exc_gp',GPe_input(i),'prob_syn_gp2snr',0.35); % initial conductance of gp to snr synapses 

spk_gp = [];
spk_snr=[];
dt=0.0001; % 0.1ms integration steps

for l =1:10
[Vm_gp, Vm_snr] = BGdelayline_excitation('I_exc_snr',80,'I_exc_gp',GPe_input(i),'prob_syn_gp2snr',0.35,'g_gp2snr_i',g_gp2snr,'n',100,'connectivity','random'); % only SNr layer. 100 neurons
spk_snr = [spk_snr; Vm_snr==15];
spk_gp = [spk_gp ; Vm_gp ==15];
end

binWidth = 100; % bin size, scale of 0.1ms

t_bar = 1:binWidth:length(spk_snr);
psth_snr = zeros(1,length(t_bar));
psth_gp = zeros(1,length(t_bar));

for psth_i = 1:length(t_bar)-1

    psth_snr(psth_i) = sum(sum(spk_snr(:,t_bar(psth_i):(t_bar(psth_i)+binWidth-1))));
    psth_gp(psth_i) = sum(sum(spk_gp(:,t_bar(psth_i):(t_bar(psth_i)+binWidth-1))));
    
end

figure(i)
subplot(3,1,1)
bar((binWidth/2+t_bar-1)*0.0001,psth_gp/(size(spk_gp,1)*(binWidth*0.0001)))
ylabel('gp PSTH (spikes/s)')
ylim([0 150])
vline(1)
xlim([0.9 1.2])
xticks([0.9 1 1.1 1.2])
xticklabels({'-100','0','100','200'})
title(sprintf('input to GPe: %d pA, net excitation to SNr = 80pA',GPe_input(i)))

subplot(3,1,2)
plotRaster(spk_snr);
vline(1)
ylabel('SNr cell')
xlabel('')
xlim([0.9 1.2])
xticks([0.9 1 1.1 1.2])
xticklabels({'-100','0','100','200'})


subplot(3,1,3)
bar((binWidth/2+t_bar-1)*0.0001,psth_snr/(size(spk_snr,1)*(binWidth*0.0001)))
xlim([0.9 1.2])
xticks([0.9 1 1.1 1.2])
xticklabels({'-100','0','100','200'})
ylim([0 150])
vline(1)
ylabel('snr PSTH (spikes/s)')
xlabel('Time (ms)')
end